In [37]:
import os

path = "/content/drive/MyDrive/NLP/DeeBERT"
os.chdir(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!pip install -r ./requirements.txt
!pip install -r ./examples/requirements.txt


### Download GLUE dataset


In [39]:
!python download_glue_data.py --data_dir glue_data --tasks all


	Completed!
	Completed!
Processing MRPC...
	Completed!
	Completed!
	Completed!
	Note (12/10/20): This script no longer downloads SNLI. You will need to manually download and format the data to use SNLI.
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
Processing MRPC...
	Completed!


### Variables


In [40]:
PATH_TO_DATA="glue_data"
MODEL_TYPE="bert"  # bert or roberta
MODEL_SIZE="base"  # base or large
DATASET="MRPC"  # SST-2, MRPC, RTE, QNLI, QQP, or MNLI

### This is for fine-tuning and evaluating models as in the original BERT paper.


In [41]:
!bash scripts/train.sh {PATH_TO_DATA} {MODEL_TYPE} {MODEL_SIZE} {DATASET}


2021-11-29 02:59:27.406445: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
11/29/2021 02:59:27 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/29/2021 02:59:27 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
11/29/2021 02:59:27 - INFO - transformers.configuration_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "mrpc",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_ran

### This is for fine-tuning DeeBERT models.


In [42]:
!bash scripts/train_highway.sh {PATH_TO_DATA} {MODEL_TYPE} {MODEL_SIZE} {DATASET}


2021-11-29 03:10:33.551077: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
11/29/2021 03:10:34 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/29/2021 03:10:34 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
11/29/2021 03:10:34 - INFO - transformers.configuration_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "mrpc",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_ran

### This is for evaluating each exit layer for fine-tuned DeeBERT models.


In [43]:
!bash scripts/eval_highway.sh {PATH_TO_DATA} {MODEL_TYPE} {MODEL_SIZE} {DATASET}


2021-11-29 03:31:04.218416: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
11/29/2021 03:31:04 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/29/2021 03:31:04 - INFO - transformers.configuration_utils -   loading configuration file ./saved_models/bert-base/MRPC/two_stage/config.json
11/29/2021 03:31:04 - INFO - transformers.configuration_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "mrpc",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  

### This is for evaluating fine-tuned DeeBERT models, given a number of different early exit entropy thresholds.


In [44]:
!bash scripts/eval_entropy.sh {PATH_TO_DATA} {MODEL_TYPE} {MODEL_SIZE} {DATASET}


0
2021-11-29 03:34:22.330725: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
11/29/2021 03:34:22 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/29/2021 03:34:22 - INFO - transformers.configuration_utils -   loading configuration file ./saved_models/bert-base/MRPC/two_stage/config.json
11/29/2021 03:34:22 - INFO - transformers.configuration_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "mrpc",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,


In [1]:
import os
import numpy as np
relative_path = "plotting/saved_models/"+MODEL_TYPE+"-"+MODEL_SIZE+"/"+DATASET+"/two_stage/"
print(relative_path)

for path, lists, frame in os.walk(relative_path):
    for value in frame:
        print(path+value)
        data = np.load(path+"/"+value, allow_pickle=True)
        print(data)
        print()


plotting/saved_models/bert-base/MRPC/two_stage/each_layer.npy
[0.81222707 0.81222707 0.81222707 0.81885125 0.83950617 0.82847896
 0.8381877  0.83788122 0.84006462 0.85667752 0.88107203 0.90086957]

plotting/saved_models/bert-base/MRPC/two_stage/entropy_0.0.npy
[{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 408}
 13.458903312683105 1.0 0.9008695652173914]

plotting/saved_models/bert-base/MRPC/two_stage/entropy_0.001.npy
[{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 408}
 13.309908628463745 1.0 0.9008695652173914]

plotting/saved_models/bert-base/MRPC/two_stage/entropy_0.005.npy
[{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 408}
 13.593218088150024 1.0 0.9008695652173914]

plotting/saved_models/bert-base/MRPC/two_stage/entropy_0.01.npy
[{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 408}
 13.405313730239868 1.0 0.9008695652173914]

plotting/saved_models/bert-base/MRPC/two_stage/en